In [1]:
from Skeleton import Skeleton,DynamicSkeleton,ConstrainedSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time

s12 = Skeleton('BODY12.xml')
s15 = ConstrainedSkeleton('BODY15_constrained_3D.xml')
# Rajagopal
# s = DynamicSkeleton('BODY15_constrained_3D.xml')
# BSM
kps =  ['RKnee', 'LWrist', 'RHip', 'RShoulder',  'LElbow', 'LHip', 'RElbow', 'RWrist', 'LKnee', 'LShoulder', 'RAnkle', 'LAnkle']
kps_ice = ['right_knee', 'left_wrist', 'right_hip', 'right_shoulder', 'left_elbow', 'left_hip', 'right_elbow', 'right_wrist', 'left_knee', 'left_shoulder', 'right_ankle','left_ankle']
gui = nimble.NimbleGUI()
gui.serve(8080)
# sudo lsof -iTCP:8080 -sTCP:LISTEN


GUIWebsocketServer will start serving a WebSocket server on ws://localhost:8070
Web GUI serving on http://localhost:8080


In [4]:
# Qualitative analysis from ICE lab test case
import json
filename = "tmp/ice/json/interaction_3p/aggregator_source_flk.flk.json"
with open(filename) as f:
    file = json.load(f)

people = {}
last_seen = {}

outfile = file.copy()

for f in range(len(file)):
    print("frame",f)
    frame = file[f]
    # outfile[f]["kp3d"] = []
    # outfile[f]["kp2d"] = []
    # outfile[f]["track_ids"] = []
    for p in range(len(frame["track_ids"])):
        data = frame["kp3d"][p]
        
        # If there is a new id, then add a dynamic skeleton instance
        if frame["track_ids"][p] not in people.keys():
            print("new body found:",frame["track_ids"][p])
            people[frame["track_ids"][p]] = DynamicSkeleton(config='BODY15_constrained_3D.xml',
                                                            osim_file='/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/Skeleton/bsm.osim', 
                                                            max_velocity=2.5)
                                                            # osim_file='/home/emartini/Biomechanical-Model/Skeleton/bsm.osim', max_velocity=2)
            dt = 100
            
            # Adjust knee joint limits to prevent from unfeasible poses
            people[frame["track_ids"][p]].q_l[9]  += 10*np.pi/180
            people[frame["track_ids"][p]].q_l[16] += 10*np.pi/180
        
        else:
            dt = (frame["timestamp"]-last_seen[frame["track_ids"][p]])/1000
            # print("dt:",dt)

        # Extract the 12 keypoints 
        target = []
        for k in kps_ice:
            if k in data.keys():
                target.append([data[k][0],data[k][1],data[k][2]])
            else:
                target.append([np.nan,np.nan,np.nan])
        target = np.array(target)
        # print(target)
        
            
        
        s12.load_from_numpy(target,kps)
        people[frame["track_ids"][p]].load_from_BODY12(s12)
        people[frame["track_ids"][p]].estimate_confidence()
        people[frame["track_ids"][p]].estimate_scale()
        
        # print([str(j.name)+": "+str(j.confidence) for j in people[frame["track_ids"][p]].keypoints_list])
        print(people[frame["track_ids"][p]]._nimble.getBodyScales().reshape(-1,3)[:,0])
        
        people[frame["track_ids"][p]].filter(dt=dt,Q=0.1)
        
        last_seen[frame["track_ids"][p]] = frame["timestamp"]
        # gui.nativeAPI().renderSkeleton(people[frame["track_ids"][p]]._nimble)
        
        # Save joints position to a new json
        pos = people[frame["track_ids"][p]]._nimble.getJointWorldPositions(people[frame["track_ids"][p]].joints)
        pos = people[frame["track_ids"][p]].correct(pos)

        # DEBUG
        s12.load_from_numpy(pos.reshape(-1,3),kps)
        s15.load_from_BODY12(s12)


        outfile[f]["kp3d"][p] = file[f]["kp3d"][p].copy()
        outfile[f]["kp2d"][p] = file[f]["kp2d"][p].copy()
        outfile[f]["track_ids"][p] = file[f]["track_ids"][p]
        
        # outfile[f]["kp3d"].append(file[f]["kp3d"][p].copy())
        # outfile[f]["kp2d"].append(file[f]["kp2d"][p].copy())
        # outfile[f]["track_ids"].append(file[f]["track_ids"][p])
        
        
        # outfile[f]["kp3d"][-1] = {}
        # print([ file[f]["track_ids"][p1] for p1 in range(len(file[f]["track_ids"]))] )
        
        
        for k in range(len(kps_ice)):      
            outfile[f]["kp3d"][-1][kps_ice[k]] = [np.nan,np.nan,np.nan]
            outfile[f]["kp3d"][-1][kps_ice[k]][0] = pos[3*k]
            outfile[f]["kp3d"][-1][kps_ice[k]][1] = pos[3*k+1]
            outfile[f]["kp3d"][-1][kps_ice[k]][2] = pos[3*k+2]
        

frame 0
frame 1
frame 2
frame 3
frame 4
frame 5
frame 6
frame 7
frame 8
frame 9
frame 10
frame 11
new body found: 1
Setting len to 0.1
Setting neutral pos to [0.96342047 0.90457786 0.90523128 0.96342047 0.96342047 0.96342047
 0.90457786 0.90523128 0.96342047 0.96342047 0.96342047 0.96342047
 0.96342047 0.96342047 0.96342047 1.         1.         1.
 0.96342047 0.96342047 1.         1.         1.         0.96342047]
  0
0.7
  0
Setting len to 0.3
Setting neutral pos to    0
-0.6
   0
Setting len to 0.1
Setting neutral pos to   0
0.2
  0
frame 12
[0.96493708 0.90817598 0.93529154 0.96493708 0.96493708 0.96493708
 0.90817598 0.93529154 0.96493708 0.96493708 0.96493708 0.96493708
 0.96493708 0.96493708 0.96493708 1.         1.         1.
 0.96493708 0.96493708 1.         1.         1.         0.96493708]
frame 13
[0.9680105  0.91062672 0.96256577 0.9680105  0.9680105  0.9680105
 0.91062672 0.96256577 0.9680105  0.9680105  0.9680105  0.9680105
 0.9680105  0.9680105  0.9680105  1.         1.

In [6]:
# Read the json and dump
outfilename = filename.replace(".json",".bm.json")
with open(outfilename, 'w') as f:
  json.dump(outfile, f)